---
title: "Traitement `items.json`"
date: 2023-05-22
author: "Julien Rabaud"
toc: true
toc-title: "Sommaire"
toc-expand: false
page-layout: article
code-copy: true
reference-location: margin
lang: fr-FR
standalone: true
embed-resources: true
---

## Requête API

Fichier `items.json` issue de la requête API :

```
https://cime.univ-pau.fr/api/items
```

## Traitement *Python* / *Pandas*

### omekaclient.py

In [3]:
from omekaclient import OmekaClient
import json
import csv

# endpoint = 'https://cime.univ-pau.fr/api'
infile = open('items.json', encoding='utf-8')
outfile = 'output.csv'

# make API request; quit if there's an error
# response, content = OmekaClient(endpoint).get('items')
# if response.status != 200:
#    print (response.status, response.reason)
#    exit()

# turn response into dictionary, retrieve metadata
fields = []
data = json.load(infile)
for D in data:
    if D['tags']:
        tags = [ d['name'] for d in D['tags'] ]
        D['tags'] = ', '.join(tags)
    if type(D['files']) is dict:
        D['files'] = D['files']['count']
    for key in ['item_type', 'collection', 'owner']:
        if type(D[key]) is dict:
            D[key] = str(D[key]['id'])
    for d in D['element_texts']:
        k = d['element']['name']
        v = d['text']
        D[k] = v
    for k, v in list(D.items()):
        if type(v) is dict or type(v) is list:
            del D[k] 
        if type(v) is bool or type(v) is int:
            D[k] = str(v)
    for k in D.keys():
        if k not in fields: fields.append(k)

# write to CSV output file using DictWriter instance
o = open(outfile, 'w', encoding='utf-8', newline='')
c = csv.DictWriter(o, [f for f in sorted(fields)], restval='NonE', extrasaction='ignore') 
c.writeheader()
for D in data:
    c.writerow({k:v for k,v in D.items() if isinstance(v, str)})
o.close()